In [1]:
import splitfolders

In [2]:
input_folder = "ClickA/Input_Data"
output = "ClickA/Processed_Data"
splitfolders.ratio(input_folder, output, seed = 1337 , ratio = (.7, .15, .15))

Copying files: 88 files [00:00, 126.25 files/s]


In [3]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

In [4]:
img_height, img_width = (224, 224)
batch_size = 32

train_data_dir = r"ClickA\Processed_Data\train"
valid_data_dir = r"ClickA\Processed_Data\val"
test_data_dir = r"ClickA\Processed_Data\test"

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
                                  
train_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical')

valid_gen = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical')

Found 60 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [6]:
test_gen = train_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size = 1,
    class_mode = 'categorical')

Found 16 images belonging to 2 classes.


In [8]:
x, y = train_gen.next()
x.shape

(28, 224, 224, 3)

In [10]:
base_model = ResNet50(include_top = False, weights = 'imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(train_gen.num_classes, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_gen, epochs = 20, validation_data = valid_gen)

Epoch 1/20
2/2 [==============================] - 13s 6s/step - loss: 0.8060 - accuracy: 0.6587 - val_loss: 4.0378e-05 - val_accuracy: 1.0000
Epoch 2/20
2/2 [==============================] - 7s 4s/step - loss: 7.6825e-06 - accuracy: 1.0000 - val_loss: 1.2755e-05 - val_accuracy: 1.0000
Epoch 3/20
2/2 [==============================] - 8s 5s/step - loss: 0.0619 - accuracy: 0.9667 - val_loss: 2.5994e-05 - val_accuracy: 1.0000
Epoch 4/20
2/2 [==============================] - 7s 4s/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 5.9605e-08 - val_accuracy: 1.0000
Epoch 5/20
2/2 [==============================] - 7s 4s/step - loss: 2.4294e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
2/2 [==============================] - 7s 4s/step - loss: 3.4154e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 8s 4s/step - loss: 2.6491e-09 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0

In [11]:
model.save('ClickA\Saved_Model\Thesis_Test5.h5')

In [13]:
val_loss, val_acc = model.evaluate(valid_gen, verbose = 1)
print ('\nVal Accuracy:', val_acc)

1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000

Val Accuracy: 1.0


In [15]:
test_loss, test_acc = model.evaluate(test_gen, verbose = 2)
print ('\nTest Accuracy:', test_acc)

16/16 - 2s - loss: 0.0000e+00 - accuracy: 1.0000

Test Accuracy: 1.0
